# Import packages

In [ ]:
import numpy as np
import pandas as pd
import statistics  

# Create Matrix of Portfolio Weights

In [ ]:
# Number of assets: 2
# Step size: 1%  (first weights: [100%, 0%], second weights: [99%, 1%] etc.)

def portfolio_weights():
    # Matrix of weights
    weights = np.array([])
    step_size = 1
    
    for a in range(0, 101, step_size):
        for b in range(0, 101, step_size):
            if (a+b) != 100:
                continue
            else:
                weights = np.append(weights, [b, a])
    
    length = weights.shape[0]/2
    weights = weights.reshape((int(length), 2))
    return weights

# Return best portfolio for 1 month (based on the Sharpe ratio)

In [ ]:
def best_portfolio_for_one_month(weights, mr_rfr, mr_stock, mr_bond, dr_stock, dr_bond):
    weights = weights/100
    # Initialize
    p_max_sharpe = -1000
    p_max_return = -1000
    # Create vector to store 'best' portfolio results
    best_p_results = np.zeros(14)    
    for w in weights:
        new_return = w[0]*mr_stock + w[1]*mr_bond
        # Store daily portfolio returns
        p_dr = []
        # Retrieve daily portfolio returns 
        for stock, bond in zip(dr_stock, dr_bond):
            p_dr.append(w[0]*stock + w[1]*bond)
        # Calculate pportfolio risk by taking the st. dev over the daily p returns
        new_risk = statistics.stdev(p_dr) * np.sqrt(len(p_dr))
        # Convert annual rate to monhtly rate
        mr_rfr = (1 + mr_rfr) ** (1/12) - 1
        new_sharpe = (new_return - mr_rfr) / new_risk            
        if new_sharpe > p_max_sharpe:
            best_p_results[0] = w[0]
            best_p_results[1] = w[1]
            best_p_results[2] = new_return
            best_p_results[3] = new_risk
            best_p_results[4] = new_sharpe 
            p_max_sharpe = new_sharpe
            
            
        '''The rest in the below is not used or needed for the thesis, 
        but interested for me to do some additional analysis'''
        
        best_p_results[5] = mr_stock
        best_p_results[6] = mr_bond        
        best_p_results[7] = mr_rfr
        if mr_stock > mr_bond:
            best_p_results[8] = 1
        elif mr_stock == mr_bond:
            best_p_results[8] = 2
        else:
            best_p_results[8] = 0
        # Risk adjusted returns
        stock_risk = statistics.variance(dr_stock)
        bond_risk = statistics.variance(dr_bond)
        stock_rar = mr_stock/stock_risk
        bond_rar = mr_bond/bond_risk
        best_p_results[9] = stock_rar
        best_p_results[10] = bond_rar
        if stock_rar > bond_rar:
            best_p_results[11] = 1
        elif stock_rar == bond_rar:
            best_p_results[11] = 2
        else:
            best_p_results[11] = 0
        # Add results for Equally Weighted and the 60/40 portfolios
        best_p_results[12] = (0.5*mr_stock+0.5*mr_bond)
        best_p_results[13] = (0.6*mr_stock+0.4*mr_bond)

    return best_p_results

# Create list of monthly dates

In [ ]:
def list_monthly_dates(first_year, last_year):
    # Gather each yyyy-mm
    all_dates = []
    # Loop over each yyyy-mm
    for year in range(first_year, last_year+1):
        for month in range(1, 12+1):
            if len(str(month)) == 2:
                date = f"{month}/{year}"
                all_dates.append(date)
            else:
                date = f"0{month}/{year}"
                all_dates.append(date)
    return all_dates

# Retrieve best portfolio for each month

In [ ]:
#dr_file = file with daily returns for all asset classes
#mr_file = file with monthly returns for all asset classes
def all_best_portfolios(first_year, last_year, mr_rfr, mr_stock, mr_bond, dr_stock, dr_bond): 
    # Import weights
    weights = portfolio_weights()
    
    # Import return data
    mr_rfr = pd.read_csv('Index & RFR Data/' + mr_rfr)       # Monthly RFR returns
    mr_stock = pd.read_csv('Index & RFR Data/' + mr_stock)   # Monthly Stock returns
    mr_bond = pd.read_csv('Index & RFR Data/' + mr_bond)     # Monthly Bond returns
    dr_stock = pd.read_csv('Index & RFR Data/' + dr_stock)   # Daily Stock returns
    dr_bond = pd.read_csv('Index & RFR Data/' + dr_bond)     # Daily Bond returns

    # List of dates: mm-yyyy 
    dates = list_monthly_dates(first_year, last_year)
    
    # Store best portfolios
    all_results = np.zeros((len(dates), 15))   
    
    month = 0
    for mm_yyyy in dates:
        # Retrieve monthly returns for month yyyy-mm
        fil_mr_rfr = mr_rfr.loc[mr_rfr['DATE'].str.contains(mm_yyyy), 'Value'].values[0]
        fil_mr_stock = mr_stock.loc[mr_stock['DATE'].str.contains(mm_yyyy), 'Value'].values[0]
        fil_mr_bond = mr_bond.loc[mr_bond['DATE'].str.contains(mm_yyyy), 'Value'].values[0]
        # Retrieve dailly returns for month yyyy_mm
        fil_dr_stock = dr_stock[dr_stock['DATE'].str.contains(mm_yyyy)]
        fil_dr_bond = dr_bond[dr_bond['DATE'].str.contains(mm_yyyy)]
        fil_dr_stock = fil_dr_stock['Value'].tolist()
        fil_dr_bond = fil_dr_bond['Value'].tolist()

        # Return best portfolio for month yyyy-mm
        result = best_portfolio_for_one_month(weights=weights, mr_rfr=fil_mr_rfr, mr_stock=fil_mr_stock, mr_bond=fil_mr_bond, dr_stock=fil_dr_stock, dr_bond=fil_dr_bond)

        # Store results
        all_results[month, [0]] = month + 1  # Store month at first column
        all_results[month, 1:] = result # Store results
        month += 1
        print(mm_yyyy)


    # Return vector of all results    
    column_names = ["Month", "W_Stock", "W_Bond", "Portfolio Return", "Portfolio St. Dev", "Portfolio Sharpe Ratio", "Stock Index Return", "Bond Index Return", "RFR Return", "Max Return", "Risk Adjusted Return Stock Index","Risk Adjusted Return Bond Index", "Best Risk Adjusted Return", "Equally Weighted Portfolio", "60%/40% Portfolio"]
    
    # Convert to a DataFrame
    all_results = pd.DataFrame(all_results, columns=column_names, index=dates)

    # Concatenate the DataFrames along the columns axis (axis=1)
    all_results.to_csv('Retrieved Best Portfolio Weights and Statistics.csv', index=True)
    return all_results

# Call function

In [ ]:
all_best_portfolios(first_year=1990, last_year=2021, mr_rfr="3M T-Bill Monthly Data.csv", mr_stock="Stock Index Monthly Data.csv", mr_bond="Bond Index Monthly Data.csv", dr_stock="Stock Index Daily Data.csv", dr_bond="Bond Index Daily Data.csv")